# Baseline model and evaluation

This notebook uses the `sumy` sdk to summarize the judgements into 100 sentences (approximately 5000 words which is the average length of summaries in train set) using SumBasic model.

SumBasic is a "Method often used as a baseline in the literature - another one used to compare the score of the algorithms. I think you can use it if you want but it has no special advantage over the LSA or TextRank." https://github.com/miso-belica/sumy/blob/main/docs/summarizators.md

SumBasic is a simple extractive text summarization model based on the concept of word frequency and probability. It was introduced by Radev et al. in their research paper titled "SumBasic: A Simple Unsupervised Multi-document Summarization System" (2002).

# Connect to Google Drive

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


# Read in Train and Test data directly from csv

In [ ]:
# Read in CSV data
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/W266 Final Project/data/train_data.csv")
train_df_filter = train_df[['index', 'judgement','summary']]
test_df = pd.read_csv("/content/drive/MyDrive/W266 Final Project/data/test_data.csv")
test_df_filter = test_df[['index', 'judgement','summary']]

# Setup Sumy

In [ ]:
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21696 sha256=0404aecbb65a8b12d65e1a8f58247f0f37db84485421cf7e97564e35062181d1
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=97208ba4069f388d964ae9ed65a4d443d33f32600f1faf8375ebe027a30bb5b9
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=1068

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Generate summary

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.sum_basic import SumBasicSummarizer

# Initialize the summarizer with the TextRank algorithm
summarizer = SumBasicSummarizer()

baseline_output = []

print("train dataset length:", len(train_df_filter))

for judgement in train_df_filter['judgement']:
  parser = PlaintextParser.from_string(judgement, Tokenizer("english"))

  # Summarize the article and get the most important sentences
  summary = summarizer(parser.document, 100)  # You can change the number of sentences as needed
  summary_sentences = " ".join([str(sentence) for sentence in summary])
  baseline_output.append(summary_sentences)

print("Summarization generated for:", len(baseline_output))

train dataset length: 7723
Summarization generated for: 7723


In [ ]:
import csv

output_file_path = '/content/drive/MyDrive/W266 Final Project/output/train_data_baseline_summary.csv'

with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

    # Write the header row (optional, if you want to include column headers)
    csv_writer.writerow(['Index', 'Summary', 'BaselineSummary'])

    # Write the data rows (judgement and its corresponding summary)
    for i in range(len(baseline_output)):
        # Write the judgement and its summary to the CSV file
        csv_writer.writerow([train_df_filter["index"][i], train_df_filter['summary'][i], baseline_output[i]])

print("Summary sentences written to:", output_file_path)

Summary sentences written to: /content/drive/MyDrive/W266 Final Project/output/train_data_baseline_summary.csv


# Evaluate results

In [ ]:
import pandas as pd
baseline_output = pd.read_csv("/content/drive/MyDrive/W266 Final Project/output/train_data_baseline_summary.csv")

In [ ]:
baseline_output.head()

,Index,Summary,BaselineSummary
0,uksc-2009-0034.txt,Mr and Mrs Agbaje were married for 38 years.\n...,The wife has been living in England continuous...
1,uksc-2009-0037.txt,"The Appellants brother, who is now deceased (t...",It was a conditional suspended possession orde...
2,uksc-2009-0048.txt,RTS specialises in the supply of automated mac...,This is the judgment of the court. The judge d...
3,uksc-2009-0031.txt,This appeal concerns the principles to be appl...,The husband cross petitioned in November. Mean...
4,uksc-2009-0018.txt,In response to various incidents of internatio...,"It affects third parties too, including the sp..."


In [ ]:
!pip install -q evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.4 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Using cached rouge_score-0.1.2-py3-none-any.whl


In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=baseline_output["BaselineSummary"],
                        references=baseline_output["Summary"])
print(results)

{'rouge1': 0.43141278545901385, 'rouge2': 0.2884847798543292, 'rougeL': 0.2610166865396496, 'rougeLsum': 0.38124037560233726}
